# Use Exhaustive Tree Search to Play Tic-Tac-Toe

In [1]:
import logging
import sys

import numpy as np
np.random.seed(0)
import gym

import boardgame2
from boardgame2 import BLACK, WHITE, EMPTY

logging.basicConfig(level=logging.DEBUG,
        format='%(asctime)s [%(levelname)s] %(message)s',
        stream=sys.stdout, datefmt='%H:%M:%S')

Environment

In [2]:
env = gym.make('TicTacToe-v0')
env.seed(0)
for key in vars(env):
    logging.info('%s: %s', key, vars(env)[key])
for key in vars(env.spec):
    logging.info('%s: %s', key, vars(env.spec)[key])

00:00:02 [INFO] env: <KInARowEnv<TicTacToe-v0>>
00:00:02 [INFO] _action_space: None
00:00:02 [INFO] _observation_space: None
00:00:02 [INFO] _reward_range: None
00:00:02 [INFO] _metadata: None
00:00:02 [INFO] _has_reset: False
00:00:02 [INFO] id: TicTacToe-v0
00:00:02 [INFO] entry_point: boardgame2:KInARowEnv
00:00:02 [INFO] reward_threshold: None
00:00:02 [INFO] nondeterministic: False
00:00:02 [INFO] max_episode_steps: None
00:00:02 [INFO] order_enforce: True
00:00:02 [INFO] _kwargs: {'board_shape': 3, 'target_length': 3}
00:00:02 [INFO] _env_name: TicTacToe


Agent

In [3]:
class ExhaustiveSearchAgent:
    def __init__(self, env):
        self.env = env
        self.learn()

    def learn(self):
        self.winner = {}   # str(state) -> player
        self.policy = {}   # str(state) -> action
        init_state = np.zeros_like(env.board, dtype=np.int8), BLACK
        self.search(init_state)

    def search(self, state):  # Tree Search: recursive implementation
        s = str(state)
        if s not in self.winner:  # the node has not been calculated
            self.winner[s] = self.env.get_winner(state) 
            if self.winner[s] is None:  # do not have immediate winner
                # try all next valid actions
                valid = self.env.get_valid(state)
                winner_actions = {}
                for x in range(valid.shape[0]):
                    for y in range(valid.shape[1]):
                        if valid[x, y]:
                            action = np.array([x, y])
                            next_state = self.env.get_next_state(state, action)
                            winner = self.search(next_state)
                            winner_actions[winner] = action

                # choose the best action
                _, player = state
                for winner in [player, EMPTY, -player]:
                    if winner in winner_actions:
                        action = winner_actions[winner]
                        self.policy[s] = action
                        self.winner[s] = winner
                        break
        return self.winner[s]

    def reset(self, mode=None):
        pass

    def step(self, observation, reward, done):
        s = str(observation)
        action = self.policy[s]
        return action

    def close(self):
        pass


agent = ExhaustiveSearchAgent(env=env)

In [4]:
def play_boardgame2_episode(env, agent, mode=None, verbose=False):
    observation, winner, done = env.reset(), 0, False
    agent.reset(mode=mode)
    elapsed_steps = 0
    while True:
        if verbose:
            board, player = observation
            print(boardgame2.strfboard(board))
        action = agent.step(observation, winner, done)
        if verbose:
            logging.info('step %d：player %d, action %s', elapsed_steps, player,
                    action)
        observation, winner, done, _ = env.step(action)
        if done:
            if verbose:
                board, _ = observation
                print(boardgame2.strfboard(board))
            break
        elapsed_steps += 1
    agent.close()
    return winner, elapsed_steps


winner, elapsed_steps = play_boardgame2_episode(env, agent, mode='test',
        verbose=True)
logging.info('test episode: winner = %d, steps = %d', winner, elapsed_steps)

+++
+++
+++
00:00:06 [INFO] step 0：player 1, action [2 2]
+++
+++
++o
00:00:06 [INFO] step 1：player -1, action [1 1]
+++
+x+
++o
00:00:06 [INFO] step 2：player 1, action [2 1]
+++
+x+
+oo
00:00:06 [INFO] step 3：player -1, action [2 0]
+++
+x+
xoo
00:00:06 [INFO] step 4：player 1, action [0 2]
++o
+x+
xoo
00:00:06 [INFO] step 5：player -1, action [1 2]
++o
+xx
xoo
00:00:06 [INFO] step 6：player 1, action [1 0]
++o
oxx
xoo
00:00:06 [INFO] step 7：player -1, action [0 1]
+xo
oxx
xoo
00:00:06 [INFO] step 8：player 1, action [0 0]
oxo
oxx
xoo
00:00:06 [INFO] test episode: winner = 0, steps = 8


In [5]:
for s in set(agent.winner).union(agent.policy):
    logging.info("state = %s, action = %s, winner = %s",
            s, agent.policy.get(s, None), agent.winner.get(s, None))

00:00:06 [INFO] state = (array([[-1, -1, -1],
       [ 0,  1,  0],
       [ 1,  0,  1]], dtype=int8), 1), action = None, winner = -1
00:00:06 [INFO] state = (array([[-1,  1, -1],
       [-1,  1,  0],
       [ 1,  1,  0]], dtype=int8), -1), action = None, winner = 1
00:00:06 [INFO] state = (array([[-1,  1,  0],
       [-1,  0,  0],
       [ 0,  0,  1]], dtype=int8), 1), action = [2 0], winner = 1
00:00:06 [INFO] state = (array([[ 1, -1,  0],
       [-1,  1,  1],
       [-1,  1,  0]], dtype=int8), -1), action = [2 2], winner = 0
00:00:06 [INFO] state = (array([[ 0,  0, -1],
       [-1,  0,  1],
       [ 0,  1,  0]], dtype=int8), 1), action = [2 0], winner = 0
00:00:06 [INFO] state = (array([[-1, -1,  1],
       [ 1,  0,  1],
       [ 1,  0, -1]], dtype=int8), -1), action = [1 1], winner = -1
00:00:06 [INFO] state = (array([[ 1, -1, -1],
       [-1,  0,  1],
       [-1,  1,  1]], dtype=int8), 1), action = [1 1], winner = 1
00:00:06 [INFO] state = (array([[ 1,  1,  0],
       [-1,  0,  1],

00:00:07 [INFO] state = (array([[ 1,  1, -1],
       [ 1, -1,  0],
       [-1,  1, -1]], dtype=int8), 1), action = None, winner = -1
00:00:07 [INFO] state = (array([[ 1, -1, -1],
       [ 0,  1,  1],
       [-1,  0,  0]], dtype=int8), 1), action = [2 2], winner = 1
00:00:07 [INFO] state = (array([[ 0, -1,  0],
       [ 1,  0,  1],
       [ 0, -1,  0]], dtype=int8), 1), action = [1 1], winner = 1
00:00:07 [INFO] state = (array([[ 1,  1,  0],
       [-1, -1,  0],
       [ 0,  0,  0]], dtype=int8), 1), action = [0 2], winner = 1
00:00:07 [INFO] state = (array([[ 1, -1,  1],
       [ 0, -1,  0],
       [ 1,  0, -1]], dtype=int8), 1), action = [1 0], winner = 1
00:00:07 [INFO] state = (array([[ 0,  0,  1],
       [ 1, -1,  1],
       [ 0,  0, -1]], dtype=int8), -1), action = [2 1], winner = -1
00:00:07 [INFO] state = (array([[-1,  1, -1],
       [ 1, -1,  0],
       [ 1,  0,  1]], dtype=int8), -1), action = [2 1], winner = 0
00:00:07 [INFO] state = (array([[ 0,  1, -1],
       [ 1,  0, -1],

00:00:07 [INFO] state = (array([[ 1,  0,  0],
       [ 0,  0,  0],
       [-1,  1,  0]], dtype=int8), -1), action = [1 1], winner = 0
00:00:07 [INFO] state = (array([[ 1, -1, -1],
       [ 0,  1,  1],
       [ 1, -1,  0]], dtype=int8), -1), action = [2 2], winner = 1
00:00:07 [INFO] state = (array([[-1,  1,  1],
       [ 0,  0,  0],
       [-1,  1,  0]], dtype=int8), -1), action = [1 1], winner = -1
00:00:07 [INFO] state = (array([[-1,  0,  1],
       [ 1,  0,  0],
       [ 0, -1,  0]], dtype=int8), 1), action = [1 2], winner = 1
00:00:07 [INFO] state = (array([[-1,  1,  1],
       [ 0,  0, -1],
       [ 0,  1, -1]], dtype=int8), 1), action = [1 1], winner = 1
00:00:07 [INFO] state = (array([[-1,  1,  1],
       [ 1,  0, -1],
       [-1,  1, -1]], dtype=int8), 1), action = [1 1], winner = 1
00:00:07 [INFO] state = (array([[ 0,  0,  0],
       [ 0,  0,  1],
       [ 0,  0, -1]], dtype=int8), 1), action = [2 1], winner = 0
00:00:07 [INFO] state = (array([[-1, -1,  0],
       [ 1,  1, -1]

00:00:07 [INFO] state = (array([[ 1,  0,  1],
       [-1,  0, -1],
       [ 0,  1,  0]], dtype=int8), -1), action = [1 1], winner = -1
00:00:07 [INFO] state = (array([[-1,  0,  1],
       [ 0, -1,  0],
       [ 1,  1,  0]], dtype=int8), -1), action = [2 2], winner = -1
00:00:07 [INFO] state = (array([[ 1,  0, -1],
       [-1,  0,  1],
       [ 1,  0,  0]], dtype=int8), -1), action = [2 2], winner = 0
00:00:07 [INFO] state = (array([[ 0, -1,  0],
       [ 0,  0,  1],
       [ 1, -1,  1]], dtype=int8), -1), action = [1 1], winner = -1
00:00:07 [INFO] state = (array([[ 1,  1,  1],
       [-1,  0, -1],
       [ 0,  1, -1]], dtype=int8), -1), action = None, winner = 1
00:00:07 [INFO] state = (array([[ 0,  1,  1],
       [ 0, -1,  0],
       [-1,  0,  0]], dtype=int8), 1), action = [2 2], winner = 1
00:00:07 [INFO] state = (array([[ 0,  0, -1],
       [-1,  1,  0],
       [ 1,  0,  0]], dtype=int8), 1), action = [2 2], winner = 1
00:00:07 [INFO] state = (array([[ 1,  0, -1],
       [-1,  1, 

00:00:07 [INFO] state = (array([[ 0,  1, -1],
       [ 1, -1, -1],
       [ 1,  0,  1]], dtype=int8), -1), action = [2 1], winner = 1
00:00:07 [INFO] state = (array([[ 0,  1, -1],
       [ 1,  0,  0],
       [-1,  1, -1]], dtype=int8), 1), action = [1 1], winner = 1
00:00:07 [INFO] state = (array([[ 1,  0, -1],
       [ 1, -1,  1],
       [-1,  1, -1]], dtype=int8), 1), action = None, winner = -1
00:00:07 [INFO] state = (array([[ 0, -1,  0],
       [ 1,  1,  0],
       [-1, -1,  1]], dtype=int8), 1), action = [1 2], winner = 1
00:00:07 [INFO] state = (array([[-1,  1,  1],
       [-1,  1, -1],
       [ 0,  1,  0]], dtype=int8), -1), action = None, winner = 1
00:00:07 [INFO] state = (array([[-1,  0, -1],
       [ 1,  0,  1],
       [ 1, -1,  0]], dtype=int8), 1), action = [1 1], winner = 1
00:00:07 [INFO] state = (array([[ 0,  0, -1],
       [ 0,  1,  0],
       [ 1, -1,  0]], dtype=int8), 1), action = [1 0], winner = 1
00:00:07 [INFO] state = (array([[ 1,  1,  0],
       [-1,  0,  0],
 

...

00:00:10 [INFO] state = (array([[ 0,  0,  0],
       [ 0,  1, -1],
       [ 1,  1, -1]], dtype=int8), -1), action = [0 2], winner = -1
00:00:10 [INFO] state = (array([[ 0, -1,  1],
       [-1,  1,  1],
       [-1,  1, -1]], dtype=int8), 1), action = [0 0], winner = 0
00:00:10 [INFO] state = (array([[-1,  0,  0],
       [ 1, -1,  1],
       [-1,  0,  1]], dtype=int8), 1), action = [0 2], winner = 1
00:00:10 [INFO] state = (array([[ 0,  0,  0],
       [ 1, -1,  0],
       [ 0, -1,  1]], dtype=int8), 1), action = [0 1], winner = 0
00:00:10 [INFO] state = (array([[ 1,  1,  0],
       [ 0,  0,  1],
       [-1, -1, -1]], dtype=int8), 1), action = None, winner = -1
00:00:10 [INFO] state = (array([[ 1,  0, -1],
       [-1,  1,  0],
       [-1,  1,  1]], dtype=int8), -1), action = None, winner = 1
00:00:10 [INFO] state = (array([[-1,  0,  1],
       [ 0, -1,  1],
       [-1,  1,  0]], dtype=int8), 1), action = [2 2], winner = 1
00:00:10 [INFO] state = (array([[-1,  0,  1],
       [ 0,  0,  0],


00:00:10 [INFO] state = (array([[ 1, -1, -1],
       [ 0,  0,  0],
       [ 0,  1,  1]], dtype=int8), -1), action = [2 0], winner = 1
00:00:10 [INFO] state = (array([[ 1,  0,  0],
       [-1,  1,  0],
       [ 0,  0, -1]], dtype=int8), 1), action = [0 2], winner = 1
00:00:10 [INFO] state = (array([[ 1,  0,  1],
       [-1, -1,  1],
       [ 0, -1,  0]], dtype=int8), 1), action = [2 2], winner = 1
00:00:10 [INFO] state = (array([[ 1,  1, -1],
       [ 0,  0, -1],
       [-1,  1,  1]], dtype=int8), -1), action = [1 1], winner = -1
00:00:10 [INFO] state = (array([[ 0,  0,  1],
       [ 1,  0, -1],
       [ 0,  0,  0]], dtype=int8), -1), action = [2 0], winner = 0
00:00:10 [INFO] state = (array([[-1,  0,  1],
       [ 0,  0, -1],
       [-1,  1,  1]], dtype=int8), 1), action = [1 0], winner = 0
00:00:10 [INFO] state = (array([[ 0,  0, -1],
       [ 1,  0,  0],
       [ 0,  0,  1]], dtype=int8), -1), action = [0 0], winner = 0
00:00:10 [INFO] state = (array([[-1,  0,  0],
       [ 0,  0,  0

00:00:10 [INFO] state = (array([[ 1,  0, -1],
       [ 0, -1,  0],
       [ 1, -1,  1]], dtype=int8), 1), action = [1 0], winner = 1
00:00:10 [INFO] state = (array([[ 1,  0, -1],
       [ 1, -1,  1],
       [ 0,  1, -1]], dtype=int8), -1), action = [2 0], winner = -1
00:00:10 [INFO] state = (array([[-1, -1,  1],
       [ 1,  1,  1],
       [ 0, -1,  0]], dtype=int8), -1), action = None, winner = 1
00:00:10 [INFO] state = (array([[ 0, -1, -1],
       [ 1,  1, -1],
       [ 0,  0,  1]], dtype=int8), 1), action = [0 0], winner = 1
00:00:10 [INFO] state = (array([[ 0, -1,  0],
       [ 1,  0, -1],
       [ 0,  0,  1]], dtype=int8), 1), action = [2 0], winner = 1
00:00:10 [INFO] state = (array([[ 0, -1,  0],
       [ 0,  0,  0],
       [ 1,  1, -1]], dtype=int8), 1), action = [0 2], winner = 0
00:00:10 [INFO] state = (array([[-1, -1,  1],
       [-1,  1,  1],
       [ 0,  1, -1]], dtype=int8), 1), action = [2 0], winner = 1
00:00:10 [INFO] state = (array([[ 0,  1,  0],
       [-1, -1,  1],


00:00:11 [INFO] state = (array([[ 1, -1, -1],
       [ 1,  1,  1],
       [-1,  0,  0]], dtype=int8), -1), action = None, winner = 1
00:00:11 [INFO] state = (array([[-1,  1,  1],
       [ 0,  0, -1],
       [-1,  0,  1]], dtype=int8), 1), action = [1 0], winner = 0
00:00:11 [INFO] state = (array([[-1,  1, -1],
       [ 0,  1, -1],
       [ 0,  1,  1]], dtype=int8), -1), action = None, winner = 1
00:00:11 [INFO] state = (array([[ 1,  1,  1],
       [-1,  0, -1],
       [ 1,  0, -1]], dtype=int8), -1), action = None, winner = 1
00:00:11 [INFO] state = (array([[-1,  0, -1],
       [ 1,  1, -1],
       [ 1,  0,  1]], dtype=int8), -1), action = [0 1], winner = -1
00:00:11 [INFO] state = (array([[-1,  0,  0],
       [-1,  0,  1],
       [ 1,  0,  1]], dtype=int8), -1), action = [2 1], winner = 1
00:00:11 [INFO] state = (array([[ 0,  0,  1],
       [ 1, -1, -1],
       [ 1,  1, -1]], dtype=int8), -1), action = [0 0], winner = -1
00:00:11 [INFO] state = (array([[ 0, -1,  0],
       [ 0,  1,  0

00:00:11 [INFO] state = (array([[-1,  1, -1],
       [-1,  0,  1],
       [ 1,  1,  0]], dtype=int8), -1), action = [2 2], winner = 1
00:00:11 [INFO] state = (array([[ 0,  1,  1],
       [ 1, -1,  0],
       [-1, -1,  0]], dtype=int8), 1), action = [2 2], winner = 1
00:00:11 [INFO] state = (array([[ 1, -1,  1],
       [ 0, -1,  0],
       [-1,  0,  1]], dtype=int8), 1), action = [1 2], winner = 1
00:00:11 [INFO] state = (array([[ 0,  0,  0],
       [ 1,  0,  0],
       [ 1, -1,  0]], dtype=int8), -1), action = [2 2], winner = 1
00:00:11 [INFO] state = (array([[ 1,  0,  0],
       [ 0,  0,  1],
       [-1, -1,  1]], dtype=int8), -1), action = [1 1], winner = 1
00:00:11 [INFO] state = (array([[ 1, -1,  1],
       [-1,  0,  0],
       [ 0,  0,  1]], dtype=int8), -1), action = [2 1], winner = 1
00:00:11 [INFO] state = (array([[-1,  0,  1],
       [ 1,  0,  0],
       [ 1, -1, -1]], dtype=int8), 1), action = [1 1], winner = 1
00:00:11 [INFO] state = (array([[-1,  0, -1],
       [ 1, -1,  1]